In [ ]:
pip install xlsxwriter

In [ ]:
pip install openpyxl 


In [ ]:
pip install -U sentence-transformers


In [ ]:
import sys
sys.path.append('../input')
from xuexibao.utils.fileUtils import *

In [ ]:
lines = readExcelToList('../input/data-app/app3/Facebook/1.xlsx')
lines2 = readExcelToList('../input/data-app/app3/Facebook/2.xlsx')[1:]
lines3 = readExcelToList('../input/data-app/app3/Facebook/3.xlsx')[1:]
lines4 = readExcelToList('../input/data-app/app3/Facebook/4.xlsx')[1:]
for i in range(len(lines)):
    lines[i].extend([1])
for i in range(len(lines2)):
    lines2[i].extend([2])
for i in range(len(lines3)):
    lines3[i].extend([3])
for i in range(len(lines4)):
    lines4[i].extend([4])
lines.extend(lines2)
lines.extend(lines3)
lines.extend(lines4)
lines = lines[1:]

In [ ]:
for i in range(len(lines)):
    lines[i][-1] = str(lines[i][-1]).strip().replace('!','').strip('.').replace('\n','').replace('\t','').replace('"','').replace('|','')
    lines[i][-2] = str(lines[i][-2]).strip().replace('!','').strip('.').replace('\n','').replace('\t','').replace('"','').replace('|','')
    lines[i][1] = str(lines[i][1]).strip().replace('!','').strip('.').replace('\n','').replace('\t','').replace('"','').replace('|','')

In [ ]:
for i in range(len(lines)):
    title = str(lines[i][3]).strip('').strip('.').replace('!','').replace('\n','')+'. '
    lines[i][4] = title + str(lines[i][4])

In [ ]:
for i in range(len(lines)):
    lines[i][4] = lines[i][4].strip().replace('!','').replace('\n','').replace('-----------------------------------------------------------------------------------------','').replace('\t','')

In [ ]:
def f1(t):
    temp = ''
    for i in t:
        if ord(i)<1200:
            temp+=i
    return temp

In [ ]:
for i in range(1,len(lines)):
    lines[i][4] = f1(str(lines[i][4]))
    lines[i][3] = f1(str(lines[i][3]))

review = {
    time:xxx,
    title:xxx,
    rating:xxx,
    reviewer:xxx,
    text:xxx,
    is_deleted:True,
    dev_reply:xxx
}

In [ ]:
arr = []
for i in lines:
    review = {
    'time':i[0],
    'guojia':i[-1],
    'title':i[3].strip().replace('!','').strip('.').replace('\n','').replace('\t','').replace('"',''),
    'rating':i[2],
    'reviewer':i[1],
    'text':i[4],
    'is_deleted':None
    'dev_reply':None
}
    arr.append(review)
    


In [ ]:
docx = [i['text'] for i in arr]
time = [i['time'] for i in arr]
rating = [i['rating'] for i in arr]
title = [i['title'] for i in arr]
guojia = [i['guojia'] for i in arr]

In [ ]:
from sentence_transformers import SentenceTransformer, util
#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

In [ ]:
def find_coment(query,docs,path):
    #Encode query and documents
    query_emb = model.encode(query)
    doc_emb = model.encode(docs)

    #Compute dot score between query and all document embeddings
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
    guojia = [i['guojia'] for i in arr]
    doc_score_pairs = list(zip(docs, scores,title,rating,time,guojia))

    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

    f = open(path,'w')
    f.write('id,socre,text,title,rating,time,guojia\n')
    count = 0
    for doc, score,a,b,c,guojia in doc_score_pairs:
        count+=1
        f.write('{},{},"{}","{}","{}","{}","{}"\n'.format(count,score,doc,a,b,c,guojia))

    f.close()

In [ ]:
f1 = readStrFromFile('../input/data-app/app3/Facebook/1.txt')

f1 = f1.replace('\r\n','').replace('\xa0',' ')

In [ ]:
find_coment(f1,docx,'./Facebook.csv')